In [ ]:
# Imports
from fbprophet import Prophet
import pandas as pd
from pathlib import Path


In [18]:
# Load Bitcoin Price Data
bitcoin_df = pd.read_csv(Path('../resources/BTC-USD.csv'))
bitcoin_df = bitcoin_df[['Date', 'Open']]
bitcoin_df = bitcoin_df.rename(columns={'Date':'ds', 'Open':'y'})
bitcoin_df['ds'] = pd.to_datetime(bitcoin_df['ds'])

ds    2022-02-08 00:00:00
y             43854.65234
Name: 127, dtype: object

In [ ]:
# Create Train/Test Data
train=bitcoin_df[(bitcoin_df['ds'] <= '2022-02-08')]
test=bitcoin_df[(bitcoin_df['ds'] > '2022-02-08')]
train.tail()

In [17]:

m = Prophet()


AttributeError: 'Series' object has no attribute 'nonzero'